## Semana 2 Día 3

Ahora profundizaremos en los siguientes puntos:

1. Diferentes modelos

2. Salidas estructuradas

3. Barreras de seguridad

In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

In [ ]:
load_dotenv(override=True)

In [ ]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"La clave de la API de OpenAI existe y comienza con {openai_api_key[:8]}")
else:
    print("OpenAI API Key no establecida")

if google_api_key:
    print(f"Google API Key existe y comienza con {google_api_key[:2]}")
else:
    print("Google API Key no establecida (y esto es opcional)")

if deepseek_api_key:
    print(f"DeepSeek API Key existe y comienza con {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key no establecida (y esto es opcional)")

if groq_api_key:
    print(f"Groq API Key existe y comienza con {groq_api_key[:4]}")
else:
    print("Groq API Key no establecida (y esto es opcional)")

In [4]:
instructions1 = "Eres un agente de ventas que trabaja para ComplAI, \
una empresa que ofrece una herramienta SaaS para garantizar el cumplimiento de SOC2 y prepararse para auditorías, impulsada por IA. \
Escribes correos electrónicos profesionales y serios."

instructions2 = "Eres un agente de ventas atractivo y entretenido que trabaja para ComplAI, \
una empresa que ofrece una herramienta SaaS para garantizar el cumplimiento de SOC2 y prepararse para auditorías, impulsada por IA. \
Escribes correos electrónicos ingeniosos y atractivos que probablemente obtengan respuesta."

instructions3 = "Eres un agente de ventas muy activo que trabaja para ComplAI, \
una empresa que ofrece una herramienta SaaS para garantizar el cumplimiento de SOC2 y prepararse para auditorías, impulsada por IA. \
Escribes correos electrónicos concisos y directos."

### Es fácil utilizar cualquier modelo con puntos finales compatibles con OpenAI

In [5]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [6]:

deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

In [7]:
sales_agent1 = Agent(name="Agente de ventas DeepSeek", instructions=instructions1, model=deepseek_model)
sales_agent2 =  Agent(name="Agente de ventas Gemini", instructions=instructions2, model=gemini_model)
sales_agent3  = Agent(name="Agente de ventas Llama3.3",instructions=instructions3,model=llama3_3_model)

In [8]:
description = "Escribe un correo electrónico de ventas en frío"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [9]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Envía un correo electrónico con el asunto indicado y el cuerpo HTML a todos los clientes potenciales de ventas. """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("juangabriel@frogames.es")  # Cambiar a tu remitente verificado
    to_email = To("juangabriel@frogames.es")  # Cambiar a tu receptor
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [10]:
subject_instructions = "Puedes escribir un asunto para un correo electrónico de ventas en frío. \
Se te proporciona un mensaje y necesitas escribir un asunto para un correo electrónico que probablemente obtenga respuesta."

html_instructions = "Puedes convertir un cuerpo de correo electrónico de texto a un cuerpo de correo electrónico HTML. \
    Se te proporciona un cuerpo de correo electrónico de texto que puede tener algún markdown \
    y necesitas convertirlo a un cuerpo de correo electrónico HTML con un diseño simple, claro y atractivo."

subject_writer = Agent(name="Escritor de asunto de correo electrónico", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Escribe un asunto para un correo electrónico de ventas en frío")

html_converter = Agent(name="Convertidor de cuerpo de correo electrónico a HTML", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convierte un cuerpo de correo electrónico de texto a un cuerpo de correo electrónico HTML")

In [11]:
email_tools = [subject_tool, html_tool, send_html_email]

In [12]:
instructions ="Eres un formateador y remitente de correos electrónicos. \
Recibes el cuerpo de un correo electrónico para enviarlo. \
Primero usas la herramienta subject_writer para escribir un asunto para el correo electrónico, \
luego usas la herramienta html_converter para convertir el cuerpo a HTML. \
Finalmente, usas la herramienta send_html_email para enviar el correo electrónico con el asunto y el cuerpo HTML."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convierte un correo electrónico a HTML y lo envía")

In [13]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [14]:
sales_manager_instructions = "Eres un gerente de ventas que trabaja para ComplAI. Utilizas las herramientas que se te proporcionan para generar correos electrónicos de ventas en frío. \
Nunca generas correos electrónicos de ventas tú mismo; siempre utilizas las herramientas. \
Pruebas las 3 herramientas del agente de ventas al menos una vez antes de elegir la mejor. \
Puedes usar las herramientas múltiples veces si no estás satisfecho con los resultados del primer intento. \
Seleccionas el mejor correo electrónico usando tu propio criterio sobre cuál será más efectivo. \
Después de elegir el correo electrónico, transfieres al agente Email Manager para formatear y enviar el correo."


sales_manager = Agent(
    name="Manager de ventas",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Envíe un correo electrónico de ventas en frío dirigido a 'Estimado director ejecutivo' desde Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

## Revisa la traza:

https://platform.openai.com/traces

In [15]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Revisión de nombre",
    instructions="Revisa si el usuario está incluyendo el nombre personal de alguien en lo que quiere que hagas.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

In [16]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [ ]:
careful_sales_manager = Agent(
    name="Manager de ventas",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

message = "Envíe un correo electrónico de ventas en frío dirigido a 'Estimado director ejecutivo' desde Alice"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

## Comrpueba la traza:

https://platform.openai.com/traces

In [20]:

message = "Envía un correo electrónico de ventas frío dirigido al Estimado CEO del Jefe de Desarrollo Comercial"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Ejercicio</h2>
            <span style="color:#ff7800;">• Prueba diferentes modelos<br/>• Agrega más barreras de entrada y salida<br/>• Usa salidas estructuradas para la generación de correos electrónicos
            </span>
        </td>
    </tr>
</table>